In [13]:
from pathlib import Path
import shutil
import json

def proc_session(data):
    res = {
        'serial': data.get('deviceInfo', {}).get('description', {}).get('Serial'),
        'start_utc': data.get('sessionInfo', {}).get('startUTCUnixTimestamp'),
        'end_utc': data.get('sessionInfo', {}).get('endUTCUnixTimestamp')
    }

    return res

def proc_user_activity(data):
    res = {}
    for item in data:
        activity = item.get("activity", "")
        times = item.get("times", [{}])[0] 
        res[f"{activity}_start"] = times.get("start_time")
        res[f"{activity}_end"] = times.get("end_time")
    return res

def proc_folder(folder_path, targ_path, extra):
    res = {}
    
    folder_path = Path(folder_path)
    targ_path = Path(targ_path)
    
    for subfolder in folder_path.iterdir():
        if subfolder.is_dir():
            new_name = str(extra['exp']) + '_' + subfolder.name.replace(' ', '_')
            
            source_file = subfolder / 'session_eeg.h5'
            target_file = targ_path / f"{new_name}.h5"
            
            shutil.copy(str(source_file), str(target_file))
            
            user_activity_file = subfolder / 'user_activity.json'
            if user_activity_file.exists():
                with open(user_activity_file) as f:
                    user_activity_data = proc_user_activity(json.load(f))
            else:
                user_activity_data = {}
            
            session_file = subfolder / 'session.dat'
            if session_file.exists():
                with open(session_file, 'r') as f:
                    session_data = proc_session(json.load(f))
            else:
                session_data = {}
            
            metadata = {**user_activity_data, **session_data, **extra}
            
            res[new_name] = metadata
    
    return res
        

In [14]:
h5_path = '/Users/ksc/Desktop/Нейротемка/code_local/Уточнение психофизиологических состояний 2023/all_h5'
exp1_path = "/Users/ksc/Desktop/Нейротемка/code_local/Уточнение психофизиологических состояний 2023/1_Расслабление и вовлеченность/Записанные данные"
exp2_path = "/Users/ksc/Desktop/Нейротемка/code_local/Уточнение психофизиологических состояний 2023/2_Тревожность и стресс/Записанные данные"
exp3_path = "/Users/ksc/Desktop/Нейротемка/code_local/Уточнение психофизиологических состояний 2023/3_Усталость и сильная усталость/Записанные данные"


e1 = proc_folder(exp1_path, h5_path, {'exp': 1})
e2 = proc_folder(exp2_path, h5_path, {'exp': 2})
e3 = proc_folder(exp3_path, h5_path, {'exp': 3})

res = {**e1, **e2, **e3}
res 

{'1_17.10_Вячеслав': {'1_start': 1697541168922729,
  '1_end': 1697541472316547,
  '2_start': 1697541519227488,
  '2_end': 1697542311524396,
  '3_start': 1697542373530698,
  '3_end': 1697543297739858,
  '4_start': 1697543367044571,
  '4_end': 1697543674306510,
  'calibrationiapf_start': 1697541081846589,
  'calibrationiapf_end': 1697541111894754,
  'calibrationbaseline_start': 1697541119436945,
  'calibrationbaseline_end': 1697541156236309,
  'serial': '820049',
  'start_utc': 1697541061073841,
  'end_utc': 1697543675290557,
  'exp': 1},
 '1_24.10_Иван': {'1_start': 1698133841634532,
  '1_end': 1698134155225199,
  '2_start': 1698134228333235,
  '2_end': 1698135062335461,
  '3_start': 1698135108213685,
  '3_end': 1698136045505583,
  '4_start': 1698136063186370,
  '4_end': 1698136409357649,
  'calibrationiapf_start': 1698133732265839,
  'calibrationiapf_end': 1698133762385003,
  'calibrationbaseline_start': 1698133769198769,
  'calibrationbaseline_end': 1698133807275470,
  'serial': '8200

In [17]:
import pandas as pd

df = pd.DataFrame.from_dict(res, orient='index').reset_index()
df.rename(columns={'index': 'filename_h5'}, inplace=True)
df.to_parquet('metadata.parquet', index=False)

In [18]:
df

,filename_h5,1_start,1_end,2_start,2_end,3_start,3_end,4_start,4_end,calibrationiapf_start,calibrationiapf_end,calibrationbaseline_start,calibrationbaseline_end,serial,start_utc,end_utc,exp,5_start,5_end
0,1_17.10_Вячеслав,1697541168922729,1697541472316547,1697541519227488,1697542311524396,1697542373530698,1697543297739858,1697543367044571,1697543674306510,1697541081846589,1697541111894754,1697541119436945,1697541156236309,820049,1.697541e+15,1.697544e+15,1,NaN,NaN
1,1_24.10_Иван,1698133841634532,1698134155225199,1698134228333235,1698135062335461,1698135108213685,1698136045505583,1698136063186370,1698136409357649,1698133732265839,1698133762385003,1698133769198769,1698133807275470,820049,1.698133e+15,1.698136e+15,1,NaN,NaN
2,1_26.10_Тамирлан,1698331242662479,1698331506349676,1698331578550598,1698332383643063,1698332450374903,1698333079999870,1698333109630452,1698333433867331,1698331105603392,1698331135778404,1698331140728236,1698331177960215,820049,1.698331e+15,1.698333e+15,1,NaN,NaN
3,1_17.10_Дарья,1697547246604420,1697547525895814,1697547579720206,1697548384437192,1697548424984544,1697549057179910,1697549108934938,1697549426388998,1697547151490835,1697547181562859,1697547184872435,1697547222484947,820049,1.697546e+15,1.697549e+15,1,NaN,NaN
4,1_19.10_Жасмина,1697717666041936,1697717978952485,1697718049367329,1697718867467382,1697719071425005,1697720025398443,1697720063078421,1697720374951540,1697717561398617,1697717591451679,1697717603399601,1697717641071340,820049,1.697717e+15,1.697720e+15,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,3_27.11_Татьяна,1701070747806313,1701071052476251,1701071223996728,1701072956137994,1701073024979561,1701074779216708,1701074862758601,1701076620463455,1701070606883831,1701070636939139,1701070658666029,1701070695494174,820049,1.701070e+15,1.701077e+15,3,1.701077e+15,1.701077e+15
62,3_09.11_Дарья,1699526603550185,1699526899387358,1699527038578084,1699528798860474,1699528851522459,1699530637140293,1699530703555478,1699532456289465,1699526515924223,1699526546032073,1699526552715826,1699526588925301,820049,1.699526e+15,1.699533e+15,3,NaN,NaN
63,3_17.11_Жасмина,1700207228665973,1700207538730477,1700207653170896,1700209439294088,1700209471419240,1700211273505841,1700211326130152,1700213167336964,1700207023664163,1700207053664826,1700207060512127,1700207106099488,820049,1.700207e+15,1.700214e+15,3,1.700213e+15,1.700214e+15
64,3_27.11_Мария,1701089007243282,1701089310609342,1701089391809473,1701091127320115,1701091175850591,1701092944441097,1701093057690441,1701094842676442,1701088892891631,1701088922997897,1701088927956120,1701088966012866,820049,1.701089e+15,1.701095e+15,3,1.701095e+15,1.701095e+15
